# Case File Generator for PyPower

In [2]:
import os
from pathlib import Path

BASE_DIR = Path.cwd()   

In [3]:
import pandas as pd

def save_zone_peak_loads(load_file, output_file="loads.csv"):
    """
    Finds the peak load for each zone individually (regardless of time)
    and saves the peak load per zone to a CSV.
    """

    # Load data
    df = pd.read_csv(load_file)

    # Identify zone columns
    zone_cols = [c for c in df.columns if c.startswith("PJM_")]

    # Build a results dict
    results = {
        "Zone": [],
        "Peak_Load": [],
        "Peak_Time": []
    }

    # Loop over zones and find each zone's own peak
    for zone in zone_cols:
        peak_idx = df[zone].idxmax()
        peak_load = df.loc[peak_idx, zone]
        peak_time = df.loc[peak_idx, "Time"]

        results["Zone"].append(zone)
        results["Peak_Load"].append(peak_load)
        results["Peak_Time"].append(peak_time)

    # Convert to DataFrame
    output_df = pd.DataFrame(results)

    # Save
    output_df.to_csv(output_file, index=False)

    print(f"Peak load for each zone saved to {output_file}")
    print(output_df)

    return output_df


DATA_DIR =BASE_DIR /"data/inputs/Base_Case/"

output_dir = DATA_DIR / "loads.csv"
input_dir = DATA_DIR / "load_profiles.csv"
if __name__ == "__main__":
    save_zone_peak_loads(
        input_dir,
        output_file=output_dir
    )


Peak load for each zone saved to /Users/fbasa/Library/CloudStorage/OneDrive-JohnsHopkins/Energy Systems/Project/GitHub/data/inputs/Base_Case/loads.csv
       Zone     Peak_Load  Peak_Time
0    PJM_AP   8826.333458         16
1  PJM_ATSI  12930.480709       5843
2  PJM_COMD  20740.668879       3950
3   PJM_Dom  18869.091246         52
4  PJM_EMAC  32296.160599       5677
5  PJM_PENE   2942.631999       5843
6  PJM_SMAC  13313.772840       5869
7  PJM_WMAC  10398.742446         15
8  PJM_West  36214.550883       5820


Generation and generation cost file match

In [6]:
import pandas as pd

# Load generator and fuel cost data
gens = pd.read_csv(BASE_DIR / "data/inputs/Base_Case/generators.csv")
fuel = pd.read_csv(BASE_DIR / "data/inputs/Base_Case/Fuels_data.csv")  # columns: FuelType, Cost ($/MWh)


# Use third row for constant costs
fuel_costs = fuel.iloc[2]

# Handle multi-fuel and storage
def compute_cost(fuel_str, plant_type, pmax):
    # Storage units
    if str(plant_type).lower() == 'storage':
        return 0
    # Split multiple fuels
    fuels = [f.strip() for f in str(fuel_str).split(',')]
    costs = []
    for f in fuels:
        if f not in fuel_costs.index:
            print(f"Warning: fuel type '{f}' not found in fuel cost data. Using 0.")
            costs.append(0)
        else:
            costs.append(fuel_costs[f])
    # Return weighted average cost (weighted by PMAX)
    return sum(costs) / len(costs)

# Assign fuel type for storage if missing
gens['FUEL_TYPE'] = gens['Modeled Fuels']
gens.loc[gens['PlantType'].str.lower() == 'storage', 'FUEL_TYPE'] = 'Storage'

# Assign cost per generator
gens['COST'] = gens.apply(lambda row: compute_cost(row['FUEL_TYPE'], row['PlantType'], row['Capacity (MW)']), axis=1)

# Fill PMIN = 0
gens['PMIN'] = 0

# Group by Zone and Fuel Type
grouped = gens.groupby(['Region Name', 'FUEL_TYPE'], as_index=False).agg({
    'Capacity (MW)': 'sum',
    'COST': 'mean'  # simple average cost
})

# Prepare MATPOWER CSV
matpower_df = pd.DataFrame()
matpower_df['GEN_ID'] = ['GEN_' + str(i+1) for i in range(len(grouped))]
matpower_df['PLANT_NAME'] = grouped['FUEL_TYPE'] + '_' + grouped['Region Name']
matpower_df['ZONE'] = grouped['Region Name']
matpower_df['FUEL_TYPE'] = grouped['FUEL_TYPE']
matpower_df['BUS'] = 0  # placeholder
matpower_df['PG'] = 0
matpower_df['PMAX'] = grouped['Capacity (MW)']
matpower_df['PMIN'] = 0
matpower_df['COST'] = grouped['COST']

# Save to CSV
matpower_df.to_csv(BASE_DIR / 'data/inputs/Base_Case/matpower_generators_combined.csv', index = False)

print("MATPOWER generator CSV created with generators combined per fuel type per zone!")

MATPOWER generator CSV created with generators combined per fuel type per zone!


# Select Data to Run

In [28]:
import pandas as pd
import numpy as np
from pypower.api import runopf, ppoption

# -----------------------------
# File paths
# -----------------------------
loads_file = DATA_DIR/"loads.csv"
network_file =DATA_DIR/ "Network.csv"
gens_file = DATA_DIR/ "matpower_generators_combined.csv"

# -----------------------------
# Load data
# -----------------------------
loads = pd.read_csv(loads_file)
network = pd.read_csv(network_file)
gens = pd.read_csv(gens_file)
load_dict = dict(zip(loads["Zone"], loads["Peak_Load"]))
# -----------------------------
# Explicit 9 zones
# -----------------------------
zones = ['PJM_AP', 'PJM_ATSI', 'PJM_COMD', 'PJM_Dom', 
         'PJM_EMAC', 'PJM_PENE', 'PJM_SMAC', 'PJM_WMAC', 'PJM_West']
bus_map = {zone: i+1 for i, zone in enumerate(zones)}  # bus numbers 1-9

# -----------------------------
# Peak load row
# -----------------------------
# print(peak_load_row)
gen_per_bus = gens.groupby('ZONE')['PMAX'].sum()
print(gen_per_bus)
slack_bus_zone = gen_per_bus.idxmax()
# -----------------------------
# Create bus data
# -----------------------------
bus_data = []
for i, zone in enumerate(zones):
    bus_num = i+1
    Pd = float(load_dict.get(zone,0))
    Qd = 0
    bus_type = 3 if zone == slack_bus_zone else 1
    bus_data.append([bus_num, bus_type, Pd, Qd, 0, 0, 1, 1.0, 0, 0, 0, 0])
bus_data = np.array(bus_data)
print(bus_data)

# -----------------------------
# Generator data
# -----------------------------
gens['ZONE'] = gens['ZONE'].str.strip()
gens = gens[gens['ZONE'].isin(zones)]  # only valid zones

gen_data = []
gen_data = []
for _, row in gens.iterrows():
    bus_num = int(bus_map[row['ZONE']])
    PG = float(row['PG'])
    QG = 0.0
    QMAX = float(row['QMAX']) if 'QMAX' in row else 0.0
    QMIN = float(row['QMIN']) if 'QMIN' in row else 0.0
    VG = 1.0
    MBASE = 100.0
    GEN_STATUS = 1
    PMAX = float(row['PMAX'])
    PMIN = float(row['PMIN'])
    PC1 = 0.0
    PC2 = 0.0
    QC1MIN = 0.0
    QC1MAX = 0.0

    gen_data.append([bus_num, PG, QG, QMAX, QMIN, VG, MBASE, GEN_STATUS, PMAX, PMIN, PC1, PC2, QC1MIN, QC1MAX])

gen_data = np.array(gen_data, dtype=float)
print("gen_data shape:", gen_data.shape)  # Should be (n_gen, 14)


# -----------------------------
# Branch data
# -----------------------------
branch_data = []
for _, row in network.iterrows():
    fbus = bus_map[row['Start_Zone'].strip()]
    tbus = bus_map[row['End_Zone'].strip()]
    r = 0.1  # small impedance
    x = 0.1
    b = 0.0
    rateA = row['Line_Max_Flow_MW']
    branch_data.append([fbus, tbus, r, x, b, rateA, rateA, rateA, 0, 0, 1])
branch_data = np.array(branch_data)

# gen_data shape: [n_gen, 13] in your setup
# last column is cost
# -----------------------------
# Generator cost (linear)
# -----------------------------
gencost_data = []
for _, row in gens.iterrows():
    # Assuming linear cost only: cost_per_MWh
    cost = float(row['COST'])
    gencost_data.append([2, 0, 0, 2, cost, 0])



# -----------------------------
# Build PyPower case
# -----------------------------
ppc = {
    'version': '2',
    'baseMVA': 100,
    'bus': bus_data,
    'gen': gen_data,
    'branch': branch_data,
    'gencost':  np.array(gencost_data),   
}

num_gen = ppc['gen'].shape[0]


ZONE
PJM_AP      12088.1
PJM_ATSI    10209.2
PJM_COMD    29157.7
PJM_Dom     27098.9
PJM_EMAC    35555.8
PJM_PENE    10149.5
PJM_SMAC     7612.0
PJM_WMAC    18302.7
PJM_West    47329.5
Name: PMAX, dtype: float64
[[1.00000000e+00 1.00000000e+00 8.82633346e+03 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.00000000e+00 1.00000000e+00 1.29304807e+04 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.00000000e+00 1.00000000e+00 2.07406689e+04 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.00000000e+00 1.00000000e+00 1.88690912e+04 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.00000000e+00 1.00000000e+00 3.229616

In [ ]:
print(ppc)

{'version': '2', 'baseMVA': 100, 'bus': array([[1.00000000e+00, 1.00000000e+00, 8.82633346e+03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 1.29304807e+04, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.00000000e+00, 1.00000000e+00, 2.07406689e+04, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.00000000e+00, 1.00000000e+00, 1.88690912e+04, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.00000000e+00, 1.00000000e+00, 3.22961606e+04, 0.00000000e+00,
        0.00000000e+

: 

# Run OPF and Reliability Index Calc

In [9]:
ppopt = ppoption(OUT_ALL=0, PF_DC=True)  # suppress detailed output
results = runopf(ppc, ppopt)

# print(ppc)

import numpy as np
import pandas as pd

# ==========================================
# 1. GENERATION RELIABILITY (Reserve Margin)
# ==========================================

# Extract Total Generation Capacity (Sum of PMAX)
# results['gen'] column 8 is PMAX (using 0-based index, PMAX is often col 8 in pypower/matpower)
# Note: Double check your ppc format. Usually PMAX is col 8, PG is col 1.
total_capacity = np.sum(results['gen'][:, 8]) 

# Extract Total Load (Real Power Demand)
# results['bus'] column 2 is Pd (Real Power Demand)
total_load = np.sum(results['bus'][:, 2])

# Calculate Reserve Margin
reserve_margin = ((total_capacity - total_load) / total_load) * 100

print(f"--- Reliability Indices ---")
print(f"Total Capacity: {total_capacity:.2f} MW")
print(f"Total Load:     {total_load:.2f} MW")
print(f"Reserve Margin: {reserve_margin:.2f}%")


# ==========================================
# 2. TRANSMISSION RELIABILITY (Line Loading)
# ==========================================

# Extract Flows and Limits
# results['branch'] column 13 is PF (Power Flow from bus) - approximate flow magnitude
flows = np.abs(results['branch'][:, 13])

# results['branch'] column 5 is RATE_A (Long term rating/limit)
limits = results['branch'][:, 5]

# Calculate Utilization for each line
# Filter out lines with 0 limit (infinite capacity or dummy lines) to avoid division by zero
valid_lines = limits > 0
utilization = (flows[valid_lines] / limits[valid_lines]) * 100

# Statistics
avg_loading = np.mean(utilization)
max_loading = np.max(utilization)
overloaded_lines = np.sum(utilization > 100)
critical_lines = np.sum(utilization > 90) # Lines near limit

print(f"\n--- Transmission Security ---")
print(f"Average Line Loading: {avg_loading:.2f}%")
print(f"Max Line Loading:     {max_loading:.2f}%")
print(f"Overloaded Lines:     {overloaded_lines}")
print(f"Critical Lines (>90%):{critical_lines}")

# Create a warning if system is stressed
if reserve_margin < 15:
    print("\n[WARNING] Reserve Margin is low (<15%). System may be vulnerable.")
if overloaded_lines > 0:
    print("\n[WARNING] Transmission violations detected.")
else:
    print("\n[INFO] System is operating within normal reliability parameters.")

PYPOWER Version 5.1.18, 10-Apr-2025 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!
--- Reliability Indices ---
Total Capacity: 197503.40 MW
Total Load:     156532.43 MW
Reserve Margin: 26.17%

--- Transmission Security ---
Average Line Loading: 48.98%
Max Line Loading:     100.00%
Overloaded Lines:     0
Critical Lines (>90%):8

[INFO] System is operating within normal reliability parameters.


In [10]:
import numpy as np

bus_numbers = bus_data[:, 0]  # bus_i
Pd_values = bus_data[:, 2]    # Pd

for bus, Pd in zip(bus_numbers, Pd_values):
    if Pd > 0:
        print(f"Bus {int(bus)} has load: {Pd} MW")
    else:
        print(f"Bus {int(bus)} has no load")


Bus 1 has load: 8826.33345765554 MW
Bus 2 has load: 12930.480709128 MW
Bus 3 has load: 20740.6688787858 MW
Bus 4 has load: 18869.0912462251 MW
Bus 5 has load: 32296.1605987098 MW
Bus 6 has load: 2942.63199893434 MW
Bus 7 has load: 13313.7728404934 MW
Bus 8 has load: 10398.7424457127 MW
Bus 9 has load: 36214.5508828388 MW


In [11]:
gen_buses = gen_data[:, 0]  # bus_i
unique_gen_buses = np.unique(gen_buses)

for bus in bus_numbers:
    if bus in unique_gen_buses:
        print(f"Bus {int(bus)} has generation")
    else:
        print(f"Bus {int(bus)} has no generation")


Bus 1 has generation
Bus 2 has generation
Bus 3 has generation
Bus 4 has generation
Bus 5 has generation
Bus 6 has generation
Bus 7 has generation
Bus 8 has generation
Bus 9 has generation


In [12]:
print(gens.columns)


Index(['GEN_ID', 'PLANT_NAME', 'ZONE', 'FUEL_TYPE', 'BUS', 'PG', 'PMAX',
       'PMIN', 'COST'],
      dtype='object')


In [13]:
 # Print generator mapping to buses
for i, row in gens.iterrows():
    bus_num = bus_map.get(row['ZONE'], None)
    if bus_num is None:
        print(f"Generator {row['PLANT_NAME']} has invalid zone: {row['ZONE']}")
    else:
        print(f"Generator {row['PLANT_NAME']} assigned to Bus {bus_num}")


Generator Bituminous_PJM_AP assigned to Bus 1
Generator Distillate Fuel Oil_PJM_AP assigned to Bus 1
Generator Energy Storage_PJM_AP assigned to Bus 1
Generator Hydro_PJM_AP assigned to Bus 1
Generator Landfill Gas_PJM_AP assigned to Bus 1
Generator Natural Gas_PJM_AP assigned to Bus 1
Generator Natural Gas, Distillate Fuel Oil_PJM_AP assigned to Bus 1
Generator Solar_PJM_AP assigned to Bus 1
Generator Waste Coal_PJM_AP assigned to Bus 1
Generator Wind_PJM_AP assigned to Bus 1
Generator Biomass_PJM_ATSI assigned to Bus 2
Generator Bituminous_PJM_ATSI assigned to Bus 2
Generator Distillate Fuel Oil_PJM_ATSI assigned to Bus 2
Generator Energy Storage_PJM_ATSI assigned to Bus 2
Generator Fossil Waste_PJM_ATSI assigned to Bus 2
Generator Hydro_PJM_ATSI assigned to Bus 2
Generator Landfill Gas_PJM_ATSI assigned to Bus 2
Generator Natural Gas_PJM_ATSI assigned to Bus 2
Generator Natural Gas, Distillate Fuel Oil_PJM_ATSI assigned to Bus 2
Generator Non-Fossil Waste_PJM_ATSI assigned to Bus 2


In [14]:
for i, bus in enumerate(bus_data):
    Pd = bus[2]
    print(f"Bus {int(bus[0])} has load Pd={Pd} MW")


Bus 1 has load Pd=8826.33345765554 MW
Bus 2 has load Pd=12930.480709128 MW
Bus 3 has load Pd=20740.6688787858 MW
Bus 4 has load Pd=18869.0912462251 MW
Bus 5 has load Pd=32296.1605987098 MW
Bus 6 has load Pd=2942.63199893434 MW
Bus 7 has load Pd=13313.7728404934 MW
Bus 8 has load Pd=10398.7424457127 MW
Bus 9 has load Pd=36214.5508828388 MW


In [15]:
import networkx as nx

G = nx.Graph()

# Add buses as nodes
for bus in bus_data[:, 0]:
    G.add_node(int(bus))

# Add branches as edges
for branch in branch_data:
    fbus = int(branch[0])
    tbus = int(branch[1])
    G.add_edge(fbus, tbus)

# Check connected components
components = list(nx.connected_components(G))
print(f"Number of connected components: {len(components)}")
for i, comp in enumerate(components):
    print(f"Component {i+1}: {comp}")


Number of connected components: 1
Component 1: {1, 2, 3, 4, 5, 6, 7, 8, 9}


In [16]:
bus_loads = bus_data[:, 2]  # Pd column
total_load = np.sum(bus_loads)
total_gen_max = np.sum(gens['PMAX'])
print("Total system load:", total_load)
print("Total maximum generation:", total_gen_max)


Total system load: 156532.43305848347
Total maximum generation: 197503.4


In [17]:
if 'COST' in gens.columns:
    print("Generator costs exist for all generators.")
else:
    print("Missing generator costs! You need to add gencost for each generator.")


Generator costs exist for all generators.


In [18]:
import networkx as nx

G = nx.Graph()
for fbus, tbus in branch_data[:, :2]:
    G.add_edge(int(fbus), int(tbus))

print("Is the network fully connected?", nx.is_connected(G))


Is the network fully connected? True


In [19]:
slack_bus_index = np.where(ppc['bus'][:, 1] == 3)[0]  # if type=3 is slack in PYPOWER bus type
print("Slack bus:", slack_bus_index)


Slack bus: [8]


In [20]:
for i, g in enumerate(ppc['gen']):
    if g[1] > g[8] or g[8]==0:
        print(f"Gen {i} on bus {g[0]} has invalid limits: PG={g[1]}, PMAX={g[8]}")


In [21]:
print("Bus connectivity check:")
connected_buses = set(ppc['branch'][:, [0,1]].flatten())
all_buses = set(ppc['bus'][:,0])
print("Disconnected buses:", all_buses - connected_buses)


Bus connectivity check:
Disconnected buses: set()


In [22]:
if np.any(ppc['branch'][:,2:4]==0):
    print("Warning: Some branches have zero impedance")


In [23]:
if 'gencost' in ppc:
    print("gencost exists, shape:", ppc['gencost'].shape)
else:
    print("gencost is missing!")


gencost exists, shape: (116, 6)


In [24]:
print("Gen shape:", ppc['gen'].shape)
print("GenCost shape:", ppc['gencost'].shape)


Gen shape: (116, 25)
GenCost shape: (116, 6)


In [25]:
print("PMAX min/max:", ppc['gen'][:, 8].min(), ppc['gen'][:, 8].max())


PMAX min/max: 1.5 15317.6


RUN DC OPF

In [26]:
# -----------------------------
# Run DC-OPF
# -----------------------------
ppopt = ppoption(OUT_ALL=0, PF_DC=True )  # suppress detailed output
results = runopf(ppc, ppopt)

# Check OPF convergence first
if results['success']:
    print("OPF converged successfully!\n")

    # Generator outputs
    gen_cols = ['Bus', 'PG', 'PMAX']
    gen_out = results['gen'][:, [0, 1, 8]]
    print("Generator Outputs (MW):")
    print(pd.DataFrame(gen_out, columns=gen_cols))

    # Total generation cost
    print("\nTotal Generation Cost ($/hr):")
    print(results['f'])

    # Branch flows
    branch_cols = ['FromBus','ToBus','FlowMW']
    branch_flows = results['branch'][:, [0, 1, 13]]
    print("\nBranch Flows (MW):")
    print(pd.DataFrame(branch_flows, columns=branch_cols))
else:
    print("OPF did not converge.\n")
    print("Check slack bus, generation limits, and network connectivity.")


PYPOWER Version 5.1.18, 10-Apr-2025 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!
OPF converged successfully!

Generator Outputs (MW):
     Bus            PG    PMAX
0    1.0  5.391000e+03  5391.0
1    1.0  7.010467e-09    18.1
2    1.0  5.850000e+01    58.5
3    1.0  1.294000e+02   129.4
4    1.0  2.760000e+01    27.6
..   ...           ...     ...
111  9.0  3.176923e-08   505.0
112  9.0  3.985000e+03  3985.0
113  9.0  2.380000e+02   238.0
114  9.0  3.988800e+03  3988.8
115  9.0  3.082900e+03  3082.9

[116 rows x 3 columns]

Total Generation Cost ($/hr):
283652.6479333622

Branch Flows (MW):
    FromBus  ToBus       FlowMW
0       1.0    2.0   700.922366
1       1.0    4.0  -100.000000
2       1.0    6.0  -765.304854
3       1.0    7.0  1100.000000
4       1.0    9.0 -1268.695623
5       2.0    1.0  -700.922366
6       2.0    9.0 -1969.617989
7       3.0    9.0   980.000000
8       4.0    1.0   100.000000
9       4.0    7.0  1200.000

In [27]:

# ==========================================
# 1. GENERATION RELIABILITY (Reserve Margin)
# ==========================================

# Extract Total Generation Capacity (Sum of PMAX)
# results['gen'] column 8 is PMAX (using 0-based index, PMAX is often col 8 in pypower/matpower)
# Note: Double check your ppc format. Usually PMAX is col 8, PG is col 1.
total_capacity = np.sum(results['gen'][:, 8]) 

# Extract Total Load (Real Power Demand)
# results['bus'] column 2 is Pd (Real Power Demand)
total_load = np.sum(results['bus'][:, 2])

# Calculate Reserve Margin
reserve_margin = ((total_capacity - total_load) / total_load) * 100

print(f"--- Reliability Indices ---")
print(f"Total Capacity: {total_capacity:.2f} MW")
print(f"Total Load:     {total_load:.2f} MW")
print(f"Reserve Margin: {reserve_margin:.2f}%")


# ==========================================
# 2. TRANSMISSION RELIABILITY (Line Loading)
# ==========================================

# Extract Flows and Limits
# results['branch'] column 13 is PF (Power Flow from bus) - approximate flow magnitude
flows = np.abs(results['branch'][:, 13])

# results['branch'] column 5 is RATE_A (Long term rating/limit)
limits = results['branch'][:, 5]

# Calculate Utilization for each line
# Filter out lines with 0 limit (infinite capacity or dummy lines) to avoid division by zero
valid_lines = limits > 0
utilization = (flows[valid_lines] / limits[valid_lines]) * 100

# Statistics
avg_loading = np.mean(utilization)
max_loading = np.max(utilization)
overloaded_lines = np.sum(utilization > 100)
critical_lines = np.sum(utilization > 90) # Lines near limit

print(f"\n--- Transmission Security ---")
print(f"Average Line Loading: {avg_loading:.2f}%")
print(f"Max Line Loading:     {max_loading:.2f}%")
print(f"Overloaded Lines:     {overloaded_lines}")
print(f"Critical Lines (>90%):{critical_lines}")

# Create a warning if system is stressed
if reserve_margin < 15:
    print("\n[WARNING] Reserve Margin is low (<15%). System may be vulnerable.")
if overloaded_lines > 0:
    print("\n[WARNING] Transmission violations detected.")
else:
    print("\n[INFO] System is operating within normal reliability parameters.")

--- Reliability Indices ---
Total Capacity: 197503.40 MW
Total Load:     156532.43 MW
Reserve Margin: 26.17%

--- Transmission Security ---
Average Line Loading: 48.98%
Max Line Loading:     100.00%
Overloaded Lines:     0
Critical Lines (>90%):8

[INFO] System is operating within normal reliability parameters.
